In [1]:
# Set up parameters for image generation
width=256
height=256
positive_signal_level = 0.1 # how much signal to add for positive and negative observations
negative_signal_level = 0.1 # how much signal to add for positive and negative observations
noise_level = 0.1 # how much noise to add
error_probability = 0.1 # the probability of making an error for individual points

NameError: name 'error_probability' is not defined

In [ ]:
# plotting

from matplotlib import pyplot as plt
%matplotlib inline
fig = plt.figure()
plt.title("Test figure")
plt.plot([1, 2], [2, 1])

In [ ]:
# Import an image!
import matplotlib.image as mpimg
img=mpimg.imread('../../data/fmri/mri-test1.png')
plt.imshow(img)
x_dimension = img.shape[0]
y_dimension = img.shape[1]

In [ ]:
# Add noise to it
import numpy as np
noise = np.random.normal(size=img.shape, scale=noise_level)
plt.imshow(img + noise)

In [ ]:
#OK, let us now use this idea to create two types of images.
def generate_image(xloc, yloc, signal, noise, src_img):
    img= src_img.copy() + np.random.normal(size=src_img.shape, scale=noise)
    img[xloc:xloc+30,yloc:yloc+30] += signal
    return img

plt.figure(1)
plt.imshow(generate_image(150, 130, signal=signal_level, noise=noise_level, src_img=img))
plt.figure(2)
plt.imshow(generate_image(160, 100, signal=signal_level, noise=noise_level, src_img=img))

In [2]:
# Create a few example images
n_subjects = 100
n_positive = 1
n_negative = 1

import pandas as pd
data = pd.DataFrame()
labels = np.empty(n_subjects*(n_positive + n_negative))
i = 0
for subject in range(n_subjects):
    positive_x = np.int(160 + np.random.random_sample()*30)
    positive_y = np.int(100 + np.random.random_sample()*30)
    negative_x = np.int(150 + np.random.random_sample()*30)
    negative_y = np.int(130 + np.random.random_sample()*30)
    for k in range(n_positive):
        X = generate_image(positive_x, positive_y, signal=positive_signal_level, noise=noise_level, src_img=img)
        data = data.append(pd.Series(X.reshape(-1)), ignore_index=True)
        labels[i] = 1; i+=1
    for k in range(n_negative):
        X = generate_image(negative_x, negative_y, signal=positive_signal_level, noise=noise_level, src_img=img)
        data = data.append(pd.Series(X.reshape(-1)), ignore_index=True)
        labels[i] = 0; i+=1

# add a new column with the labels
#full_data = data
#full_data['label'] = labels

ImportError: No module named pandas

In [ ]:
n_examples = data.shape[0]
total_positive = 0
total_negative = 0
mean_positive = np.empty([x_dimension, y_dimension])
mean_negative = np.empty([x_dimension, y_dimension])

for i in range(n_examples):
    example = data.iloc[[i]].values.reshape(x_dimension, y_dimension)
    if (labels[i]==1):        
        mean_positive += example
        total_positive +=1
    else:
        mean_negative += example
        total_negative +=1
        
mean_positive /= total_positive
mean_negative /= total_negative


In [ ]:
plt.figure(1); plt.imshow(mean_negative-mean_positive)
mean_negative - mean_positive

In [ ]:
import scipy.stats

tail = scipy.stats.norm.interval(1 - error_probability, scale=noise_level / n_examples)
i = 0
